<a href="https://colab.research.google.com/github/nicolay-r/arekit-googletrans-sampler/blob/master/arekit_googletrans_sampler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nicolay-r/arekit-googletrans-sampler

Cloning into 'arekit-googletrans-sampler'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 146 (delta 79), reused 115 (delta 48), pack-reused 0
Receiving objects: 100% (146/146), 575.50 KiB | 11.28 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [9]:
!sudo pip uninstall arekit
!sudo pip install git+https://github.com/nicolay-r/AREkit.git@master

Found existing installation: arekit 0.23.0
Uninstalling arekit-0.23.0:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/arekit-0.23.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/arekit/*
Proceed (y/n)? Y
  Successfully uninstalled arekit-0.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/nicolay-r/AREkit.git (to revision master) to /tmp/pip-req-build-b0ftcop7
  Running command git clone -q https://github.com/nicolay-r/AREkit.git /tmp/pip-req-build-b0ftcop7
  Created wheel for arekit: filename=arekit-0.23.0-py3-none-any.whl size=209429 sha256=2825261ae65eaf2599b80c8c3bc98dba45332b3149a82fa5df29158c171052b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-rmynfya6/wheels/14/fd/94/ef61d15cd459cf509ced698654c749df2b6e844d8e33c1e662
Successfully built arekit


In [5]:
%cd "arekit-googletrans-sampler"
!sudo pip instal 

[Errno 2] No such file or directory: 'arekit-googletrans-sampler'
/content/arekit-googletrans-sampler
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/nicolay-r/AREkit (to revision 0.23.0-rc) to /tmp/pip-req-build-t4sdbynl
  Running command git clone -q https://github.com/nicolay-r/AREkit /tmp/pip-req-build-t4sdbynl
  Running command git checkout -b 0.23.0-rc --track origin/0.23.0-rc
  Switched to a new branch '0.23.0-rc'
  Branch '0.23.0-rc' set up to track remote branch '0.23.0-rc' from 'origin'.


In [6]:
!python -m arekit.download_data

100% 53.5k/53.5k [00:00<00:00, 431kB/s]
100% 705k/705k [00:00<00:00, 1.49MB/s]
100% 54.2k/54.2k [00:00<00:00, 3.00MB/s]
100% 57.4k/57.4k [00:00<00:00, 2.07MB/s]
100% 737/737 [00:00<00:00, 344kB/s]
100% 3.40M/3.40M [00:00<00:00, 18.8MB/s]
100% 3.46M/3.46M [00:00<00:00, 25.2MB/s]
100% 27.6M/27.6M [00:00<00:00, 53.1MB/s]
100% 146M/146M [00:01<00:00, 98.2MB/s]
100% 52.1M/52.1M [00:00<00:00, 84.2MB/s]
100% 321M/321M [00:03<00:00, 94.6MB/s]
435kB [00:00, 1.37MB/s]
434kB [00:00, 2.06MB/s]
435kB [00:00, 1.30MB/s]


In [10]:
!sudo python translate.py --writer csv --source rusentrel --sampler bert --dest_lang en --docs_limit 1

Loading RuSentRel Collection: 100%|█████████████████████████████████████████████| 1253/1253 [00:02<00:00, 520.82opins/s]
INFO:arekit.common.data.storages.base:Rows calculation process started. [Required by Pandas-Based storage kernel]
Calculating rows count (sample [DataType.Train]): 682rows [00:01, 343.76rows/s]
INFO:arekit.common.data.storages.base:Filling with blank rows: 682
INFO:arekit.common.data.storages.base:Completed!
100%|████████████████████████████████████████████████████████████████████████████████▊| 680/682 [00:35<00:00, 19.00it/s]
INFO:arekit.contrib.utils.data.writers.csv_pd:Saving... 682: _out/serialize-bert-rusentrel-en-1/sample-train-0.tsv.gz
INFO:arekit.contrib.utils.data.writers.csv_pd:Saving completed!


Now we can take a look onto the results:

In [17]:
from os.path import join
from arekit.contrib.utils.data.readers.csv_pd import PandasCsvReader

reader = PandasCsvReader()
storage = reader.read("_out/serialize-bert-rusentrel-en-1/sample-train-0.tsv.gz")
storage._df.head()

,id,doc_id,label,text_a,s_ind,t_ind,sent_ind,entity_values,entity_types,entities
0,o0_i0_,1.0,2.0,"According to statements from Pentagon , purpos...",7.0,15.0,10.0,"Pentagon,USA,igil,rakqa","ORG,LOC,ORG,LOC","4,7,15,19"
1,o1_i0_,1.0,2.0,Why also in Moscow and Damascus who are also f...,30.0,11.0,14.0,"Moscow,Damascus,ig,Syria,washington,USA,rezakhakh","LOC,LOC,ORG,LOC,GEOPOLIT,LOC,PER","3,5,11,13,16,30,35"
2,o2_i0_,1.0,2.0,Why also in Moscow and Damascus who are also f...,13.0,30.0,14.0,"Moscow,Damascus,ig,Syria,washington,USA,rezakhakh","LOC,LOC,ORG,LOC,GEOPOLIT,LOC,PER","3,5,11,13,16,30,35"
3,o3_i0_,1.0,2.0,Anxiety USA obvious : the one who will be the ...,25.0,1.0,33.0,"USA,Aleppo,Syria,javan,Moscow,washington","LOC,PER,LOC,ORG,GEOPOLIT,GEOPOLIT","1,15,25,28,32,36"
4,o4_i0_,1.0,2.0,As an example ali-khamenei cited the situation...,22.0,30.0,63.0,"ali khamenei,Syria,Ukraine,america","PER,LOC,LOC,GEOPOLIT","3,22,26,30"
